# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse      Shipments  \
0  Apr 20 2023 10:43AM  261495        10        SO94607   
1  Apr 20 2023 10:41AM  261494        10  Clear-9397851   
2  Apr 20 2023 10:37AM  261493        19        9397849   
3  Apr 20 2023 10:36AM  261492        15        9397837   
4  Apr 20 2023 10:35AM  261490        10      PRF-58800   
5  Apr 20 2023 10:35AM  261490        10      PRF-58803   
6  Apr 20 2023 10:35AM  261490        10      PRF-58806   
7  Apr 20 2023 10:35AM  261490        10      PRF-58809   
8  Apr 20 2023 10:35AM  261490        10      PRF-58812   
9  Apr 20 2023 10:35AM  261490        10      PRF-58818   

                   Customer ShipmentStatus  
0  Senseonics, Incorporated       Released  
1            ClearSpec, LLC       Released  
2            Graystone, LLC       Released  
3      Emersa Waterbox, LLC       Released  
4                   Bio-PRF       Released  
5                   Bio-PRF       Released  
6                   Bio-PRF       Released  
7                   Bio-PRF       Released  
8                   Bio-PRF       Released  
9                   Bio-PRF       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
25  261490       Released         12
26  261492       Released          1
27  261493       Released          1
28  261494       Released          1
29  261495       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
261490                NaN        NaN      12.0
261492                NaN        NaN       1.0
261493                NaN        NaN       1.0
261494                NaN        NaN       1.0
261495                NaN        NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
261400                0.0       20.0      22.0
261406                0.0        1.0       0.0
261410                2.0        2.0       5.0
261432                0.0        0.0       1.0
261448                0.0        0.0       5.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
261400                  0         20        22
261406                  0          1         0
261410                  2          2         5
261432                  0          0         1
261448                  0          0         5

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               261400          0         20        22
1               261406          0          1         0
2               261410          2          2         5
3               261432          0          0         1
4               261448          0          0         5

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               261400                  20       22
1               261406                   1         
2               261410         2         2        5
3               261432                            1
4               261448                            5

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                  Customer
0   Apr 20 2023 10:43AM  261495        10  Senseonics, Incorporated
1   Apr 20 2023 10:41AM  261494        10            ClearSpec, LLC
2   Apr 20 2023 10:37AM  261493        19            Graystone, LLC
3   Apr 20 2023 10:36AM  261492        15      Emersa Waterbox, LLC
4   Apr 20 2023 10:35AM  261490        10                   Bio-PRF
16  Apr 20 2023 10:33AM  261488        10          Eywa Pharma Inc.
28  Apr 20 2023 10:32AM  261489        19            Graystone, LLC
29  Apr 20 2023 10:29AM  261487        19            Graystone, LLC
30  Apr 20 2023 10:28AM  261486        21         NBTY Global, Inc.
31  Apr 20 2023 10:26AM  261485        19          Methapharm, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                  Customer Completed  \
0  Apr 20 2023 10:43AM  261495        10  Senseonics, Incorporated             
1  Apr 20 2023 10:41AM  261494        10            ClearSpec, LLC             
2  Apr 20 2023 10:37AM  261493        19            Graystone, LLC             
3  Apr 20 2023 10:36AM  261492        15      Emersa Waterbox, LLC             
4  Apr 20 2023 10:35AM  261490        10                   Bio-PRF             
5  Apr 20 2023 10:33AM  261488        10          Eywa Pharma Inc.             
6  Apr 20 2023 10:32AM  261489        19            Graystone, LLC             
7  Apr 20 2023 10:29AM  261487        19            Graystone, LLC             
8  Apr 20 2023 10:28AM  261486        21         NBTY Global, Inc.             
9  Apr 20 2023 10:26AM  261485        19          Methapharm, Inc.             

  Executing Released  
0                  1  
1                  1  
2                  1  
3                  1  
4                 12  
5                 12  
6                  1  
7                  1  
8         1           
9                  3

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                  Customer Released  \
0  Apr 20 2023 10:43AM  261495        10  Senseonics, Incorporated        1   
1  Apr 20 2023 10:41AM  261494        10            ClearSpec, LLC        1   
2  Apr 20 2023 10:37AM  261493        19            Graystone, LLC        1   
3  Apr 20 2023 10:36AM  261492        15      Emersa Waterbox, LLC        1   
4  Apr 20 2023 10:35AM  261490        10                   Bio-PRF       12   
5  Apr 20 2023 10:33AM  261488        10          Eywa Pharma Inc.       12   
6  Apr 20 2023 10:32AM  261489        19            Graystone, LLC        1   
7  Apr 20 2023 10:29AM  261487        19            Graystone, LLC        1   
8  Apr 20 2023 10:28AM  261486        21         NBTY Global, Inc.            
9  Apr 20 2023 10:26AM  261485        19          Methapharm, Inc.        3   

  Executing Completed  
0                      
1                      
2                      
3                      
4                      
5                      
6                      
7                      
8         1            
9

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                  Customer Released  \
0  Apr 20 2023 10:43AM  261495        10  Senseonics, Incorporated        1   
1  Apr 20 2023 10:41AM  261494        10            ClearSpec, LLC        1   
2  Apr 20 2023 10:37AM  261493        19            Graystone, LLC        1   
3  Apr 20 2023 10:36AM  261492        15      Emersa Waterbox, LLC        1   
4  Apr 20 2023 10:35AM  261490        10                   Bio-PRF       12   

  Executing Completed  
0                      
1                      
2                      
3                      
4

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Apr 20 2023 10:43AM  261495        10  Senseonics, Incorporated       1.0   
1  Apr 20 2023 10:41AM  261494        10            ClearSpec, LLC       1.0   
2  Apr 20 2023 10:37AM  261493        19            Graystone, LLC       1.0   
3  Apr 20 2023 10:36AM  261492        15      Emersa Waterbox, LLC       1.0   
4  Apr 20 2023 10:35AM  261490        10                   Bio-PRF      12.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        NaN        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Apr 20 2023 10:43AM  261495        10  Senseonics, Incorporated       1.0   
1  Apr 20 2023 10:41AM  261494        10            ClearSpec, LLC       1.0   
2  Apr 20 2023 10:37AM  261493        19            Graystone, LLC       1.0   
3  Apr 20 2023 10:36AM  261492        15      Emersa Waterbox, LLC       1.0   
4  Apr 20 2023 10:35AM  261490        10                   Bio-PRF      12.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        0.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         1830391      33.0        0.0        0.0
15          784340      28.0       20.0        0.0
19         3137654      93.0       12.0        0.0
20          522816       5.0        3.0        2.0
21          261486       0.0        1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  1830391      33.0        0.0        0.0
1        15   784340      28.0       20.0        0.0
2        19  3137654      93.0       12.0        0.0
3        20   522816       5.0        3.0        2.0
4        21   261486       0.0        1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10      33.0        0.0        0.0
1        15      28.0       20.0        0.0
2        19      93.0       12.0        0.0
3        20       5.0        3.0        2.0
4        21       0.0        1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released   33.0
1        15  Released   28.0
2        19  Released   93.0
3        20  Released    5.0
4        21  Released    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse    10    15    19   20   21
Status                               
Completed   0.0   0.0   0.0  2.0  0.0
Executing   0.0  20.0  12.0  3.0  1.0
Released   33.0  28.0  93.0  5.0  0.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status    10    15    19   20   21
0          Completed   0.0   0.0   0.0  2.0  0.0
1          Executing   0.0  20.0  12.0  3.0  1.0
2           Released  33.0  28.0  93.0  5.0  0.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status    10    15    19   20   21
0  Completed   0.0   0.0   0.0  2.0  0.0
1  Executing   0.0  20.0  12.0  3.0  1.0
2   Released  33.0  28.0  93.0  5.0  0.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()